### Imports & Installs

In [2]:
!pip3 install openml

  Created wheel for openml: filename=openml-0.12.2-py3-none-any.whl size=137326 sha256=9ff35e6714f940a6c403fe4cee686524d0ba6b790c7d239cdf5d2990622fbd6b
  Stored in directory: c:\users\ernest\appdata\local\pip\cache\wheels\af\c1\4c\101610c3fdf2067758bb8f79c08cf9a5cf58ff561c1e19999f
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11731 sha256=e09a51a0233c5bc18381a4f1189fc7c6942ada9cb16b1e43ad4bf76f0c2749de
  Stored in directory: c:\users\ernest\appdata\local\pip\cache\wheels\a2\de\68\bf3972de3ecb31e32bef59a7f4c75f0687a3674c476b347c14
Successfully built openml liac-arff


In [4]:
import collections
import matplotlib.pyplot as plt
import numpy as np
import openml
import sklearn
%matplotlib inline

In [5]:
study_id = 123
tag = 'study_%d' % study_id
study = openml.study.get_study(study_id, 'tasks')

In [12]:
benchmark_suite = openml.study.get_suite(suite_id='OpenML100')

In [18]:
for task_id in benchmark_suite.tasks:
    task = openml.tasks.get_task(task_id)
    features, targets = task.get_X_and_y

OpenMLServerException: https://www.openml.org/api/v1/xml/data/features/1176 returned code 274: No features found. Additionally, dataset processed with error - None

In [15]:
len(benchmark_suite.tasks)

100